In [1]:
import torch
import torchvision.transforms as T
import numpy as np
import matplotlib.pyplot as plt
import torchvision.transforms.functional as F

torch.cuda.is_available()

device = torch.device("cuda:0")

torch.set_grad_enabled(False)

In [2]:
in_channels = 3

In [3]:
out_channels = 10

In [4]:
kernel_size = (2, 2)

In [5]:
conv = torch.nn.Conv2d (in_channels, out_channels, kernel_size, stride = 1 )
x = torch.randn((100, in_channels, 32, 32))
y = torch.randn((100, in_channels, 32, 32))

In [6]:
x.dim()

4

In [7]:
import math

In [8]:
def convolve(a, weight, bias, kernel, inc, outc) :
    n = a.size(0)
    stride = (1,1)
    dilation = (1,1)
    padding = (0,0)
    print(weight.view(-1, weight.size(0)).size())
    unfold_a = torch.nn.functional.unfold(a, kernel_size = kernel, stride = stride, padding = padding, dilation = dilation)
    h_out = math.floor(1 + (a.size(2) + 2 * padding[0] - dilation[0] * (kernel[0] - 1 ) - 1)/stride[0])
    w_out = math.floor(1 + (a.size(3) + 2 * padding[1] - dilation[1] * (kernel[1] - 1 ) - 1)/stride[1])
    print("Unfold size:",unfold_a.size())
    print("Weight size:",weight.view(outc,-1).size())
    print("Bias size:",bias.view(1,-1,1).size())
    print((weight.view(outc,-1) @ unfold_a).size())
    return (weight.view(outc,-1) @ unfold_a + bias.view(1,-1,1)).view(n, outc, h_out, w_out)

In [9]:
result_our_conv = convolve(x,conv.weight,conv.bias,kernel_size,in_channels,out_channels)
result_pytorch_conv = conv(x)

torch.Size([12, 10])
Unfold size: torch.Size([100, 12, 961])
Weight size: torch.Size([10, 12])
Bias size: torch.Size([1, 10, 1])
torch.Size([100, 10, 961])


In [10]:
l = list(conv.parameters())

In [11]:
l[0].size(),l[1].size()

(torch.Size([10, 3, 2, 2]), torch.Size([10]))

In [12]:
torch.testing.assert_allclose(result_pytorch_conv, result_our_conv)

In [13]:
conv = torch.nn.Conv2d (in_channels, out_channels, kernel_size, stride = 1 )
conv1 = torch.nn.Conv2d (in_channels, out_channels, kernel_size, stride = 1 )
s = torch.nn.Sequential(conv,conv1)

In [34]:
parameters = [(torch.empty(2,2).fill_(1),torch.empty(2,2).normal_())]

In [35]:
parameters

[(tensor([[1., 1.],
          [1., 1.]]),
  tensor([[-0.8852, -2.3041],
          [ 0.7805, -0.8119]]))]

In [36]:
for (p,g) in parameters:
    g.zero_()

In [37]:
parameters

[(tensor([[1., 1.],
          [1., 1.]]),
  tensor([[0., 0.],
          [0., 0.]]))]